EXTRACT

In [1]:
import pandas as pd

# EXTRACT
df = pd.read_csv("transactions_ip.csv")
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Simpan hasil mentah jika ingin
df.to_csv("/tmp/extracted_sales.csv", index=False)
df.head()

,transaction_id,customer_id,transaction_date,total_amount,payment_method,shipping_method,delivery_time,coupon_code,city,product_id,sales_factor
0,1,506,2023-07-22,11300000.0,Cash on Delivery,Tiki,1 minggu,GRATISONGKIR,Jakarta,12,0.7
1,2,23275,2024-07-01,11053000.0,DANA,Tiki,2-3 hari,SALE20,Bandung,3,0.9
2,3,24101,2022-11-01,12741000.0,Transfer Bank,JNE,4-5 hari,NaN,Bandung,5,0.8
3,4,24188,2022-02-04,4921000.0,OVO,JNE,4-5 hari,SALE20,Yogyakarta,10,0.6
4,5,33766,2022-02-06,2163000.0,Transfer Bank,GrabExpress,1 minggu,NaN,Jakarta,11,0.7


TRANSFORM

dim_date

In [2]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
dim_date = df[['transaction_date']].drop_duplicates().copy()
dim_date['date_id'] = dim_date['transaction_date'].dt.strftime('%Y%m%d').astype(int)
dim_date['year'] = dim_date['transaction_date'].dt.year
dim_date['month'] = dim_date['transaction_date'].dt.month
dim_date['day'] = dim_date['transaction_date'].dt.day

dim customer

In [3]:
dim_customer = df[['customer_id']].drop_duplicates().copy()

dim_product

In [4]:
dim_product = df[['product_id']].drop_duplicates().copy()

dim_city

In [5]:
dim_city = df[['city']].drop_duplicates().copy()
dim_city['city_id'] = dim_city.index + 1

fact_sales

In [6]:
fact_sales = df.merge(dim_date, on='transaction_date') \
               .merge(dim_city, on='city')

fact_sales = fact_sales[[
    'transaction_id', 'customer_id', 'product_id', 'date_id', 'city_id',
    'total_amount', 'payment_method', 'shipping_method',
    'delivery_time', 'coupon_code', 'sales_factor'
]]

LOAD

In [7]:
dim_date.to_csv("/tmp/dim_date.csv", index=False)
dim_customer.to_csv("/tmp/dim_customer.csv", index=False)
dim_product.to_csv("/tmp/dim_product.csv", index=False)
dim_city.to_csv("/tmp/dim_city.csv", index=False)
fact_sales.to_csv("/tmp/fact_sales.csv", index=False)